In [2]:
import yt
from matplotlib import rc_context
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
print(yt.__version__)

3.6.1


In [6]:
Norm = np.array([1.0,
  1./3.,
  1./3.,
  1./3.,
  2./3.,
  4./3.,
  4./9.,
  1./9.,
  1./9.,
  1./9.,
  2./3.,
  2./3.,
  2./3.,
  2./9.,
  2./9.,
  2./9.,
  2.0,
  4./3.,
  4./9.])
print(Norm.shape)

(19,)


In [7]:
############    modify if needed    ############################
step1 = 60400
step2 = 80400
plot_int = 10
temperature = 1e-5
alpha0 = 0.
cs2 = 1./3.
nx = 64
ny = 2
nz = 2
ncomp = 19  # nvel in LBM
###############################################################

Nframes = (step2 - step1)//plot_int + 1
fnoise_ndarray = np.zeros((Nframes, ncomp, nx, ny, nz))
gnoise_ndarray = np.zeros((Nframes, ncomp, nx, ny, nz))

plot_file_fnoise_root = "./data_mixture/lbm_data_shshan_alpha0_" + "{:.2f}".format(alpha0) +\
          "_xi_" + "{:.1e}".format(temperature) + "_size{:d}-{:d}-{:d}_continue/data_fnoise/fn".format(nx,ny,nz)
plot_file_gnoise_root = "./data_mixture/lbm_data_shshan_alpha0_" + "{:.2f}".format(alpha0) +\
          "_xi_" + "{:.1e}".format(temperature) + "_size{:d}-{:d}-{:d}_continue/data_gnoise/gn".format(nx,ny,nz)

'''
plot_file_fnoise = plot_file_fnoise_root + f"{step1:06d}"
ds0 = yt.load(plot_file_fnoise)
print(ds0.field_list)
'''

# load in noise multifab of each frame into ndarray
for n in range(step1, step2+1, plot_int):
    plot_file_fnoise = plot_file_fnoise_root + f"{n:06d}"
    plot_file_gnoise = plot_file_gnoise_root + f"{n:06d}"
    ds0 = yt.load(plot_file_fnoise)
    fnoise_data_frame = ds0.covering_grid(
    level=0, left_edge=ds0.domain_left_edge, dims=ds0.domain_dimensions)
    ds1 = yt.load(plot_file_gnoise)
    gnoise_data_frame = ds1.covering_grid(
    level=0, left_edge=ds1.domain_left_edge, dims=ds1.domain_dimensions)
    for a in range(0, ncomp):
        fnoise_ndarray[(n-step1)//plot_int, a, :,:,:] = fnoise_data_frame["boxlib", "fa{:d}".format(a)]
        gnoise_ndarray[(n-step1)//plot_int, a, :,:,:] = gnoise_data_frame["boxlib", "ga{:d}".format(a)]
    

print(fnoise_ndarray.shape)


yt : [INFO     ] 2025-02-18 05:26:22,144 Parameters: current_time              = 60400.0
yt : [INFO     ] 2025-02-18 05:26:22,146 Parameters: domain_dimensions         = [64  2  2]
yt : [INFO     ] 2025-02-18 05:26:22,149 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-02-18 05:26:22,152 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2025-02-18 05:26:22,214 Parameters: current_time              = 60400.0
yt : [INFO     ] 2025-02-18 05:26:22,215 Parameters: domain_dimensions         = [64  2  2]
yt : [INFO     ] 2025-02-18 05:26:22,216 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-02-18 05:26:22,218 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2025-02-18 05:26:23,168 Parameters: current_time              = 60410.0
yt : [INFO     ] 2025-02-18 05:26:23,169 Parameters: domain_dimensions         = [64  2  2]
yt : [INFO     ] 2025-02-18 05:26:23,171 Parameters: domain_left_edge          = [0. 0. 0

(2001, 19, 64, 2, 2)


In [10]:
tau = 1.
lbda = 1./tau
lbda_bar = lbda/(1+0.5*lbda)
factor1 = 2.*lbda_bar - lbda_bar**2
factor2 = temperature/cs2*Norm

fnoise_mode1_corv = np.zeros((ncomp, nx, ny, nz))
for a in range(1, 3+1):
    for n in range(Nframes):
        fnoise_mode1_corv[a,:,:,:] = fnoise_mode1_corv[a,:,:,:] + fnoise_ndarray[n,1,:,:,:]**2
    fnoise_mode1_corv[a,:,:,:] = fnoise_mode1_corv[a,:,:,:]/Nframes

fnoise_mode1_corv_normalized = fnoise_mode1_corv/temperature/factor1/0.5
print(fnoise_mode1_corv_normalized)

print("numerical corvariance divided by theoretical value: mean = {:.5f} and var = {:.5f}"\
       .format(np.mean(fnoise_mode1_corv_normalized), np.var(fnoise_mode1_corv_normalized)))


[[[1.00829332 1.02784092]
  [0.9832743  0.99068059]]

 [[0.97909002 0.9849608 ]
  [0.97993281 1.02474442]]

 [[0.9576478  0.99486739]
  [0.9801318  1.00659626]]

 [[1.04527674 0.94433709]
  [0.98461744 1.00057272]]

 [[0.99776697 1.00835715]
  [0.97676174 1.06885389]]

 [[0.97425134 0.9039497 ]
  [0.98263926 0.97828802]]

 [[0.97869233 1.01986694]
  [1.02442826 1.02638337]]

 [[1.00690336 0.9818385 ]
  [1.02699954 1.01308046]]

 [[1.00636707 1.01738344]
  [1.00760896 1.02777195]]

 [[1.01102456 0.93327147]
  [0.98716771 0.95230889]]

 [[1.03591211 0.94437102]
  [0.97716091 1.01799909]]

 [[1.04033778 1.01788316]
  [1.04573975 1.02955169]]

 [[0.99970464 0.9871978 ]
  [1.04513799 1.03065561]]

 [[0.91251661 0.96809607]
  [0.96082244 1.02409626]]

 [[1.00058156 1.02090425]
  [1.03072828 0.97896429]]

 [[0.96525327 1.03933877]
  [1.00127552 0.98942586]]

 [[0.9945718  1.03283031]
  [1.00751537 1.01946332]]

 [[0.97126711 1.00446289]
  [1.01402758 1.02418639]]

 [[1.01693698 1.00602193]
  